In [ ]:
# IDA Microcode

The microcode, also called IR (Intermediate Representation), is the first stage of IDA decompilation process.
In this case, microcode is the IR of assembly instruction, think of them like LLVM'S IR.